In [1]:
# Install dependencies from requirements.txt
!pip3 install -r requirements.txt

In [2]:
# Initialize Git and DVC repositories
!git init
!dvc init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /home/jupyter/week_4/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                      

In [ ]:
%%writefile .gitignore
.venv/
__pycache__/
*.pyc
.vscode/
.ipynb_checkpoints/
.env
gcloud-creds.json
.DS_Store

In [3]:
# Configure DVC to use Google Cloud Storage as remote storage
BUCKET_NAME="mlops-course-week1-001"
!dvc remote add -d gcp-store gs://$BUCKET_NAME/dvc_store

Setting 'gcp-store' as a default remote.


In [4]:
# Authenticate with Google Cloud
PATH_TO_CREDENTIALS="gcloud-creds.json"
!gcloud auth activate-service-account --key-file=$PATH_TO_CREDENTIALS

Activated service account credentials for: [github-actions-bot@buoyant-country-473106-a3.iam.gserviceaccount.com]


To take a quick anonymous survey, run:
  $ gcloud survey



In [ ]:
# Commit Initial setup and DVC remote configuration to Git
!git add .dvc/config .gitignore requirements.txt setup.ipynb
!git commit -m "Initial commit: Setup project with DVC and Git"

In [ ]:
# Download dataset file
!wget -O data.csv "https://github.com/IITMBSMLOps/ga_resources/blob/week_1/data/raw/iris.csv?raw=true"

In [ ]:
# Train and save the model
!python3 train.py

In [ ]:
# Track the model files and artifacts via DVC
!dvc add data.csv model.joblib

# Track the code files via git
!git add data.csv.dvc model.joblib.dvc .gitignore
!git commit -m "feat: Track dataset and model with DVC"

In [ ]:
# Push files and artifacts to google cloud storage via DVC
!dvc push

In [ ]:
# Add Git remote and set main branch
!git remote add origin https://github.com/mayank160920/iris-mlops-ci-1.git
!git branch -M main
# git push -u origin main

In [ ]:
# Create GitHub Actions workflow directory
!mkdir -p .github/workflows

In [ ]:
%%writefile .github/workflows/ci.yml
name: MLOps CI Pipeline

on:
  push:
    branches: [dev]
  pull_request:
    branches: [main]

permissions:
  contents: write
  pull-requests: write

jobs:
  test-and-report:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout code
        uses: actions/checkout@v3

      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.9'

      - name: Upgrade pip
        run: python -m pip install --upgrade pip

      - name: Install core dependencies
        run: |
          pip install -r requirements.txt

      - name: Run CML
        uses: iterative/setup-cml@v2

      - name: Authenticate with Google Cloud
        uses: google-github-actions/auth@v1
        with:
          credentials_json: '${{ secrets.GCP_SA_KEY }}'

      - name: Pull artifacts from DVC
        run: dvc pull

      - name: Run Pytest and Save Report
        run: pytest --maxfail=1 --disable-warnings -q > report.txt

      - name: Create CML report on Pull Request
        if: github.event_name == 'pull_request'
        env:
          repo_token: ${{ secrets.GITHUB_TOKEN }}
        run: |
          echo "## ✅ MLOps CI Report" > cml_report.md
          echo "**Branch:** ${{ github.head_ref }}" >> cml_report.md
          echo "**Commit:** \`${{ github.sha }}\`" >> cml_report.md
          echo "" >> cml_report.md
          echo "### 🧪 Test Summary" >> cml_report.md
          cat report.txt >> cml_report.md
          echo "" >> cml_report.md
          echo "---" >> cml_report.md
          echo "_Generated automatically by CML_" >> cml_report.md
          cml comment create cml_report.md

Overwriting .github/workflows/ci.yml


In [ ]:
# Commit tests, requirements, and CI workflow to Git and push to dev branch
!git add .
!git commit -m "feat: Add tests, requirements, and CI workflow"
!git checkout -b dev  # Create and switch to the dev branch

In [ ]:
!git push -u origin dev